In [1]:
import gym
import pybullet as p
import pybullet_data as pd

import time


from stable_baselines import PPO2
from stable_baselines.common.evaluation import evaluate_policy

/home/hayley/anaconda3/envs/benelot/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/hayley/anaconda3/envs/benelot/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/hayley/anaconda3/envs/benelot/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/hayley/anaconda3/envs/benelot/lib/pyt

In [2]:
pybullet_client = p
p.connect(p.GUI)
p.setAdditionalSearchPath(pd.getDataPath())

plane = p.loadURDF("plane.urdf")
p.setGravity(0, 0, -9.8)
p.setTimeStep(1. / 500)
urdfFlags = p.URDF_USE_SELF_COLLISION

p.getCameraImage(480, 320)
p.setRealTimeSimulation(0)

timeStep = 1. / 500

In [7]:
from pybullet_utils import pd_controller_stable
import math


class QuadrupedStablePD(object):

    def __init__(self, pybullet_client, timeStep, useFixedBase=True):
        self._pybullet_client = pybullet_client

        print("LOADING quadruped!")

        startPos = [0.007058990464444105, 0.03149299192130908, 0.4918981912395484]
        startOrn = [
            0.005934649695708604, 0.7065453990917289, 0.7076373820553712, -0.0027774940359030264
        ]
        self._sim_model = self._pybullet_client.loadURDF("/home/hayley/Desktop/gym/pybullet_data/laikago/laikago_toes.urdf",
                                                     startPos,
                                                     startOrn,
                                                     flags=urdfFlags,
                                                     useFixedBase=False)
    
    
        _sim_model = self._sim_model
        #print(_sim_model)
    
        
        # ------------  2,5,8 and 11 are the lower legs  -----------------------------------
        
        '''
        lower_legs = [2, 5, 8, 11]
        for l0 in lower_legs:
            for l1 in lower_legs:
                if (l1 > l0):
                    enableCollision = 1
                    print("collision for pair", l0, l1,
                    p.getJointInfo(_sim_model, l0)[12],
                    p.getJointInfo(_sim_model, l1)[12], "enabled=", enableCollision)
                p.setCollisionFilterPair(_sim_model, _sim_model, 2, 5, enableCollision)
        '''
    
        # ------------  jointoffset  -----------------------------------
        jointIds = []
        paramIds = []
        jointOffsets = []
        jointDirections = [-1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1]
        jointAngles = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

        for i in range(4):
            jointOffsets.append(0)
            jointOffsets.append(-0.7)
            jointOffsets.append(0.7)

        maxForceId = p.addUserDebugParameter("maxForce", 0, 100, 20)
        
        for j in range(p.getNumJoints(_sim_model)):
            
            # you can change the properties such as mass, friction and restitution coefficient
            p.changeDynamics(_sim_model, j, linearDamping=0, angularDamping=0)
            
            info = p.getJointInfo(_sim_model, j)
            #print(info)
            jointName = info[1]
            jointType = info[2]
            
            if (jointType == p.JOINT_PRISMATIC or jointType == p.JOINT_REVOLUTE):
                jointIds.append(j)
                
                
                
        index = 0
        for j in range(p.getNumJoints(_sim_model)):
            p.changeDynamics(_sim_model, j, linearDamping=0, angularDamping=0)
            info = p.getJointInfo(_sim_model, j)
            js = p.getJointState(_sim_model, j)
            #print(info)
            jointName = info[1]
            jointType = info[2]
            
            
            # 여기서 지금 index out of range
            if (jointType == p.JOINT_PRISMATIC or jointType == p.JOINT_REVOLUTE):
                paramIds.append(p.addUserDebugParameter(jointName.decode("utf-8"), -4, 4,
                                        (js[0] - jointOffsets[index]) / jointDirections[index]))
            index = index+1       
     
    
        
        
        
       
            
    def reset(self):
   
   
        return

    def step(self, action):
        if action == self.LEFT:
            self.agent_pos -= 1
        elif action == self.RIGHT:
            self.agent_pos += 1
        else:
            raise ValueError("Received invalid action={} which is not part of the action space".format(action))

        # Account for the boundaries of the grid
        self.agent_pos = np.clip(self.agent_pos, 0, self.grid_size)

        # Are we at the left of the grid?
        done = self.agent_pos == 0

        # Null reward everywhere except when reaching the goal (left of the grid)
        reward = 1 if self.agent_pos == 0 else 0

        # Optionally we can pass additional info, we are not using that for now
        info = {}

        return np.array(self.agent_pos).astype(np.float32), reward, done, info

    def render(self, mode='console'):
        print("")
        
    def close(self):
        pass

In [ ]:
class Camera:

    def __init__(self):
        pass

    def move_and_look_at(self, i, j, k, x, y, z):
        lookat = [x, y, z]
        distance = 10
        yaw = 10
        self._p.resetDebugVisualizerCamera(distance, yaw, -20, lookat)
        

In [8]:
env = QuadrupedStablePD(p, timeStep)

LOADING quadruped!


UnboundLocalError: local variable 'enableCollision' referenced before assignment

In [8]:
obs = env.reset()
env.render()

print(env.observation_space)
print(env.action_space)
print(env.action_space.sample())

AttributeError: 'QuadrupedStablePD' object has no attribute 'observation_space'